In [ ]:
Multiple Agents


In [7]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="llama3-groq-tool-use:latest",
    base_url="http://localhost:11434/v1",
    api_key="ollama",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)


# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

await Console(team.run_stream(task="Write a short poem about the fall season."))
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。要求中文格式。"))
await team.reset()  # Reset the team for the next run.


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
As summer's warmth begins to fade,
The fall season takes its place, displayed.
Golden leaves rustle in the air,
A tapestry rich beyond compare.

Nature's canvas, painted bright,
With hues of orange, red, and light.
Crisp cool breezes whisper through,
A soothing serenity anew.

Falling raindrops tap out a beat,
As earthy aromas start to meet.
Winter's chill is yet to come,
Yet in the fall, life is won.
---------- critic ----------
Your poem beautifully captures the essence of the fall season. The imagery is vivid and evokes a sense of coziness. The line "A tapestry rich beyond compare" is particularly striking, showcasing your skill with language. APPROVE
---------- user ----------
将这首诗用中文唐诗风格写一遍。要求中文格式。
---------- primary ----------
Here's the translation in Mandarin:

落日温度渐冷,
秋季入场展示.
橘黄叶子轻响,
绮丽的絪毯。

天然画布，精彩绽出，
红橙色调柔和明的颜色。
干爽风语在空中轻说,
宁静的气息开始演奏。

落雨滴声拍出鼓板，
大地的嗅觉开始合适。
没有寒气来临，这季节,
生命得到了胜利。
-